### Code to calculate mean profiles of LWC for different bins of Iorg 
#### author: Claudia Acquistapace
#### date; 08/02/2023



In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path
import glob
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib
from datetime import datetime, timedelta
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from matplotlib import rcParams
from warnings import warn
import datetime as dt
from scipy import interpolate
import matplotlib as mpl
import os.path
import itertools
import os.path

lat_lon_path = '/data/sat/goes-r-abi/ATLANTIC/2017-21_experiment/lat_lon_leif/'
iorg_data_df = pd.read_csv( '/work/ML_work_DC/claudia_51k_iorg.csv')
    
iorg_data = iorg_data_df.to_xarray()
print(iorg_data)

# reading the number of images in the selected file 
n_images = len(iorg_data.index.values)

# path and filename of ERA5 data
path_era5 = '/data/mod/era/era5/tropical_atlantic'
# reading one era5 data to extract height dimension (needed for defining output matrices of era5 data)
profile_era5_start = xr.open_dataset(path_era5+'/2017/03/profb_presslev_20170322T1300.nc')
height = profile_era5_start.level.values

# output path 
path_out = '/net/ostro/ML_work_DC/'

# initializing output variables
IORG = iorg_data.iorg.values
clwc_profile = np.zeros((n_images, len(height)))
clwc_profile_std = np.zeros((n_images, len(height)))
clwc_profile.fill(np.nan)
clwc_profile_std.fill(np.nan)
im_name_arr = np.asarray([''  for x in np.arange(n_images)])
id_lat_lon_arr =  np.asarray([''  for x in np.arange(n_images)])
datetime_arr = np.asarray([''  for x in np.arange(n_images)])

ind_start = len('/p/scratch/deepacf/kiste/DC/dataset/barbados/barbados_leif/1/')
print(im_name_arr)

<xarray.Dataset>
Dimensions:      (index: 51000)
Coordinates:
  * index        (index) int64 0 1 2 3 4 5 ... 50995 50996 50997 50998 50999
Data variables:
    location     (index) object '/p/scratch/deepacf/kiste/DC/dataset/barbados...
    datetime     (index) object '2017-03-19 13:06:10.200' ... '2021-12-31 15:...
    day_of_year  (index) int64 78 78 78 78 78 78 78 ... 365 365 365 365 365 365
    year         (index) int64 2017 2017 2017 2017 2017 ... 2021 2021 2021 2021
    hour         (index) int64 13 13 13 13 13 13 13 13 ... 14 14 14 14 14 14 15
    minute       (index) int64 6 6 6 6 6 21 21 21 21 ... 10 20 20 30 30 40 40 20
    iorg         (index) float64 0.7773 0.6127 0.5857 ... 0.7671 0.6495 0.7025
    tsne-2d-one  (index) float64 0.8025 0.6687 0.475 ... 0.4789 0.2564 0.4766
    tsne-2d-two  (index) float64 0.4999 0.4063 0.4236 ... 0.8031 0.3034 0.8062
    month        (index) int64 3 3 3 3 3 3 3 3 3 ... 12 12 12 12 12 12 12 12 12
['' '' '' ... '' '' '']


In [2]:
# loop on the images to collect era 5 data if they are present
for ind_images in range(n_images):

    # read image id string and id for lat/lon
    id_image = iorg_data.location.values[ind_images][ind_start:ind_start+14]
    im_name_arr[ind_images] = id_image

    id_lat_lon = iorg_data.location.values[ind_images][ind_start+14:ind_start+16]
    id_lat_lon_arr[ind_images] = id_lat_lon
    
    #print(ind_images, id_lat_lon, id_image)
    
    # reading year, month, day to build name of the image and find corresponding era5 data
    datetime_value = pd.to_datetime(iorg_data.datetime.values[ind_images])
    datetime_arr[ind_images] = datetime_value

    yy = str(datetime_value.year)
    mm = str(datetime_value.month)
    dd = str(datetime_value.day)
    hh = str(datetime_value.hour)
    mn = str(datetime_value.minute)

    # reading lat/lon
    # building era5 path
    if len(dd) == 1:
        dd = '0'+dd
    if len(hh) == 1:
        hh = '0'+hh
    if len(mm) == 1:
        mm = '0'+mm
    if len(mn) == 1:
        mn = '0'+mn

    # constructing date of the selected image (needed for era5)
    date = yy+mm+dd+hh+mn+'00'

    # assigning path for era5 based on the date
    era5_day_path = path_era5+'/'+yy+'/'+mm+'/'

    # proceed only if there are era5 data for the selected date
    if os.path.isfile(era5_day_path+'profb_presslev_'+yy+mm+dd+'T'+hh+'00.nc') * \
     os.path.isfile(era5_day_path+'surfskinvarb_'+yy+mm+dd+'T'+hh+'00.nc'):

        # find how many lat/lons we have associated to the id-image
        if id_lat_lon != '_h':
            file_lat =  lat_lon_path+id_image+'_lat'+id_lat_lon+'.npy'
            file_lon =  lat_lon_path+id_image+'_lon'+id_lat_lon+'.npy'
        else:
            file_lat =  '/net/ostro/ML_work_DC/halo_lat_lon/halo_lat.npy'
            file_lon =  '/net/ostro/ML_work_DC/halo_lat_lon/halo_lon.npy'              
        #print(file_lat, file_lon)

        # if lat and lon files are present then process the data
        if (os.path.isfile(file_lat) * os.path.isfile(file_lon)):

            # reading lats/lons for the id_image
            lat_data = np.load(file_lat)
            lon_data = np.load(file_lon)
            lat_max = np.nanmax(lat_data)
            lat_min = np.nanmin(lat_data)
            lon_min = np.nanmin(lon_data)
            lon_max = np.nanmax(lon_data)

            # reading era
            profile_era5 = xr.open_dataset(era5_day_path+'profb_presslev_'+yy+mm+dd+'T'+hh+'00.nc')
            surface_era5 = xr.open_dataset(era5_day_path+'surfskinvarb_'+yy+mm+dd+'T'+hh+'00.nc')

            # selecting the area corresponding to the crop
            surface_crop = surface_era5.where((surface_era5.latitude > lat_min)*(surface_era5.latitude <= lat_max) \
                                            * (surface_era5.longitude > lon_min) *(surface_era5.longitude <= lon_max))

            profiles_crop = profile_era5.where((profile_era5.latitude > lat_min)*(profile_era5.latitude <= lat_max) \
                                            * (profile_era5.longitude > lon_min)*(profile_era5.longitude <= lon_max))
            
            
            # saving mean value of cloud liquid water content
            clwc_profile[ind_images,:] = profiles_crop.clwc.mean(dim=('longitude', 'latitude'), skipna='True')
            clwc_profile_std[ind_images,:] = profiles_crop.clwc.std(dim=('longitude', 'latitude'), skipna='True')



ValueError: unexpected encoding parameters for 'netCDF4' backend: ['complevele']. Valid encodings are: {'_FillValue', 'contiguous', 'chunksizes', 'least_significant_digit', 'dtype', 'compression', 'shuffle', 'fletcher32', 'zlib', 'complevel'}

In [3]:
# storing data in a ncdf file for each area
crop_data = xr.Dataset(
data_vars={
    'im_names': (('n_crops',), im_name_arr, {'long_name': 'Names of image', 'units':''}),
    'datetime': (('n_crops',), datetime_arr, {'long_name': 'Names of image', 'units':''}),
     "IORG": (('n_crops',), IORG, {'long_name': 'organization index', 'units':'', "standard_name": "IORG"}),
    'clwc_std':(('n_crops','levels'), clwc_profile_std, {'long_name': 'specific cloud liquid water content standard deviation', 'units':'kg kg**-1'}),
    'clwc':(('n_crops','levels'), clwc_profile, {'long_name': 'specific cloud liquid water content', 'standard_name':'rel hum', 'units':'kg kg**-1'}),
},
coords={
    "n_crops": (('n_crops',), np.arange(n_images) ,), # leave units intentionally blank, to be defined in the encoding
    "levels": (('levels',), height, {"axis": "pressure_level","positive": "up","units": "millibars", "long_name":'pressure_level'}),
},
attrs={'CREATED_BY'     : 'Claudia Acquistapace',
                'CREATED_ON'       : str(datetime.now()),
                'FILL_VALUE'       : 'NaN',
                'PI_NAME'          : 'Claudia Acquistapace',
                'PI_AFFILIATION'   : 'University of Cologne (UNI), Germany',
                'PI_ADDRESS'       : 'Institute for geophysics and meteorology, Pohligstrasse 3, 50969 Koeln',
                'PI_MAIL'          : 'cacquist@meteo.uni-koeln.de',
                'DATA_DESCRIPTION' : 'ERA5 variables for all the crops of the selected satellite position ',
                'DATA_DISCIPLINE'  : 'Atmospheric Physics - Remote Sensing Radar Profiler',
                'DATA_GROUP'       : 'Model: reanalysis',
                'DATA_LOCATION'    : 'Atlantic Ocean - Eurec4a campaign domain',
                'DATA_SOURCE'      : 'ERA5',
                'DATA_PROCESSING'  : 'https://github.com/ClauClouds/ML_work_DC',
                'COMMENT'          : '' }
)




# assign additional attributes following CF convention
crop_data = crop_data.assign_attrs({
    "Conventions": "CF-1.8",
    "title": crop_data.attrs["DATA_DESCRIPTION"],
    "institution": crop_data.attrs["PI_AFFILIATION"],
    "history": "".join([
        "source: " + crop_data.attrs["DATA_SOURCE"] + "\n",
        "processing: " + crop_data.attrs["DATA_PROCESSING"] + "\n",
        " adapted to enhance CF compatibility\n",
    ]),  # the idea of this attribute is that each applied transformation is appended to create something like a log
    "featureType": "satellite-era5",
})

# storing ncdf data
crop_data.to_netcdf(path_out+'IORG_LWC_era5.nc', encoding={"clwc":{"zlib":True, "complevel":9}, \
                                                           "clwc_std":{"zlib":True, "complevel":9}, \
                                                           "IORG":{"zlib":True, "complevel":9}, \
                                                           "datetime":{'zlib':True, "complevel":9}})